In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm


import cv2
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
import torch.nn as nn 
import time
import torch.nn.functional as F
import torch.optim as optim 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
img = plt.imread('../input/microsoft-catsvsdogs-dataset/PetImages/Cat/0.jpg')
print(img.shape)
plt.imshow(img);

In [ ]:
img = plt.imread('../input/microsoft-catsvsdogs-dataset/PetImages/Dog/2.jpg')
print(img.shape)
plt.imshow(img);

In [ ]:
class CatsVSDogs():
    
    # Size of the image
    IMG_SIZE = 128
    
    # Directory location
    CATS = '../input/microsoft-catsvsdogs-dataset/PetImages/Cat/'
    DOGS = '../input/microsoft-catsvsdogs-dataset/PetImages/Dog/'
    
    # Labels for cats and dogs
    LABELS = {CATS:0, DOGS:1}
    
    # Initializing variables
    training_data = []
    catcount = 0
    dogcount = 0
    
    def make_training_data(self):
        for label in self.LABELS:
            
            # Looping through each pictures
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)

                    # Reading images and converting to grayscale
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

                    # Resizing images
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))

                    # Getting the training data
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    
                    # Checking distribution of data
                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                        
                except Exception as e:
                    pass

            np.random.shuffle(self.training_data)
            np.save("training_data_128.npy", self.training_data)
            print("Cates: ", self.catcount)
            print("Dogs: ", self.dogcount)
            

catsvdogs = CatsVSDogs()
catsvdogs.make_training_data()

In [ ]:
# training data augmentation
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
# testing data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None: 
            self.y = torch.Tensor(y.numpy())
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [ ]:
training_data = np.load("./training_data_128.npy", allow_pickle = True)
training_data.shape

In [ ]:
torch.Tensor(training_data[0][0]).shape

In [ ]:
# Creating a tensor from a list of numpy.ndarrays is extremely slow 
# so converting it first to np.array
X_np = np.array([i[0] for i in training_data])

# Create a tensor from the NumPy array
X = torch.from_numpy(X_np).view(-1, 128, 128)

# Scaling the features
X = X / 255.0

# Getting the target
y = torch.Tensor([i[1] for i in training_data])

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
y_label = []
real = torch.argmax(y)
for i in range(len(y)):
    real = torch.argmax(y[i])
    y_label.append(real.item())
y_label = torch.Tensor(y_label)
print('Cats :',(y_label == 0.).sum().item())
print('Dogs :',y_label.sum().item())

In [ ]:
#split train and test
val = int(len(X)*0.3)
train_x = X[:-val]
train_y = y_label[:-val]
val_x = X[-val:]
val_y = y_label[-val:]
print("Train size :",len(train_x))
print("Validation size :",len(val_x))

In [ ]:
batch_size=64

train_set = ImgDataset(train_x,train_y,train_transform)
test_set = ImgDataset(val_x,val_y,test_transform)

train_loader = DataLoader(train_set, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size= batch_size, shuffle=False)

In [ ]:
# torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
# torch.nn.MaxPool2d(kernel_size, stride, padding)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # input [1, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128), 
            nn.ReLU(), 
            nn.Linear(128,2)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)

In [ ]:
#setting
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = Classifier().to(device)
cirection = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [ ]:
#model.eval()

In [ ]:
#train
epochs = 40
v_loss = []
t_loss = []
for epoch in range(epochs):
    epoch_start_time = time.time()
    train_acc =0.0
    val_acc =0.0
    train_loss = 0.0
    val_loss =0.0
    model.train()
    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        x,y = data[0].to(device),data[1].to(device)
        # print(x.shape)
        y_pred = model(x)
        loss = cirection(y_pred,y.long())
        loss.backward()
        optimizer.step()
        train_acc +=np.sum(np.argmax(y_pred.cpu().data.numpy(),axis=1)== y.cpu().numpy())
        train_loss +=loss.item()
        
    model.eval()
    with torch.no_grad():
        for i,data in enumerate(test_loader):
            valx ,valy = data[0].to(device),data[1].to(device)
            val_pred = model(valx)
            batch_loss = cirection(val_pred,valy.long())
            val_acc +=np.sum(np.argmax(val_pred.cpu().data.numpy(),axis=1)== valy.cpu().numpy())
            val_loss +=batch_loss.item()
        t_loss.append(train_loss)    
        v_loss.append(val_loss)
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, epochs, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/test_set.__len__(), val_loss/test_set.__len__()))

In [ ]:
plt.title('Training performance')
plt.plot(t_loss, label='training loss')
plt.plot(v_loss, label='validation loss')
plt.legend()
plt.show()